In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [8]:
# Load model from Weeks 3/4 

df = pd.read_csv("course_lead_scoring.csv")

cat_features = list(df.dtypes[df.dtypes == 'object'].index)
print(cat_features)

num_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(num_features)


for col in cat_features:
    df[col] = df[col].fillna('NA')

for col in num_features:
    df[col] = df[col].fillna(0)




df.isnull().sum()



['lead_source', 'industry', 'employment_status', 'location']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']


lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [ ]:
# Check and remove 'converted' from numerical features list
if 'converted' in num_features:
    num_features.remove('converted')
    
# Check and remove 'converted' from categorical features list if it was mistakenly there
if 'converted' in cat_features:
    cat_features.remove('converted')

# # Initiate the model
all_features = cat_features +  num_features

In [ ]:
# Define functions that train / run the model 

def train(df_train, y_train, C=1.0):
    """Trains the Logistic Regression model."""
    dicts = df_train[cat_features + num_features].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    # Using solver='liblinear' is important for C < 1.0 (though here C=1.0)
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000) 
    model.fit(X_train, y_train)

    return dv, model

def predict(df, dv, model):
    """Generates probability predictions on the given DataFrame."""
    dicts = df[cat_features + num_features].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [ ]:
C = 1.0 
n_splits = 5

In [ ]:

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train): 
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.converted.values
    y_val = df_val.converted.values


NameError: name 'n_splits' is not defined